# PyCUDA installation

In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 14.4 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1.2-cp312-cp312-linux_x86_64.whl size=659050 sha256=706e984f258a8696fa427f78655f92cafbef22ff45caa4bb8ae200caf511aa77
  Stored in directory: /root/.cache/pip/wheels/d5/36/f3/ac5f09d768cad3fa15d5a3449bdfe65c3de58e69d036c73228
Successfully built pycuda




---



# Version 4: using ```ElementwiseKernel```

The first part of the code is the usual one.

In [2]:
import numpy as np

import pycuda.driver as cuda
import pycuda.autoinit
import pycuda.gpuarray as gpuarray

########
# MAIN #
########

start = cuda.Event()
end   = cuda.Event()

N = 100000

h_a = np.random.randn(N).astype(np.float32)
h_b = np.random.randn(N).astype(np.float32)

d_a = gpuarray.to_gpu(h_a)
d_b = gpuarray.to_gpu(h_b)

In this example, ```d_c``` is explicitly defined. This is necessary for the use of the ```ElementwiseKernel``` module.

In [3]:
d_c = gpuarray.empty_like(d_a)

Load the ```ElementwiseKernel``` module.

In [4]:
from pycuda.elementwise import ElementwiseKernel

The ```ElementwiseKernel``` enables to define only the kernel instructions to be elementwise executed within the kernel. Here, to generalize Version #1, a general linear combination between ```d_a``` and ```d_b``` is considered. A reference to the elementwise kernel is defined in ```lin_comb```.


In [5]:
lin_comb = ElementwiseKernel(
        "float *d_c, float *d_a, float *d_b, float a, float b",
        "d_c[i] = a * d_a[i] + b * d_b[i]")

Invoke the ```lin_comb``` function.

In [6]:
# --- Warmup execution
lin_comb(d_c, d_a, d_b, 2, 3)

start.record()
lin_comb(d_c, d_a, d_b, 2, 3)
end.record()
end.synchronize()
secs = start.time_till(end) * 1e-3
print("Processing time = %fs" % (secs))

Processing time = 0.000137s


The last part is as usual.

In [7]:
h_c = d_c.get()

if np.array_equal(h_c, 2 * h_a + 3 * h_b):
  print("Test passed!")
else :
  print("Error!")

cuda.Context.synchronize()

Test passed!
